This tutorial walks through some uses of the PyTorch implementation of NeurOps.



Let's start by importing the PyTorch implementation and some other useful packages.

In [1]:
from pytorch.src import *

import copy
import numpy as np
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We define a LeNet-style model, which has three convolutional model followed by two fully-connected model. We use the `ModSequential` class to wrap the `ModConv2d` and `ModLinear` model, which allows us to mask, prune, and grow the model. We also use the `track_activations` and `track_auxiliary_gradients` arguments to enable the tracking of activations and auxiliary gradients later. By adding the `input_shape` of the data, we can compute the conversion factor of how many input neurons to add to the first linear layer when a new output channel is added to the final convolutional layer. 

In [2]:
model = ModSequential(
        ModConv2d(in_channels=1, out_channels=8, kernel_size=7, masked=True, padding=1, learnable_mask=True),
        ModConv2d(in_channels=8, out_channels=16, kernel_size=7, masked=True, padding=1, prebatchnorm=True, learnable_mask=True),
        ModConv2d(in_channels=16, out_channels=16, kernel_size=5, masked=True, prebatchnorm=True, learnable_mask=True),
        ModLinear(64, 32, masked=True, learnable_mask=True),
        ModLinear(32, 10, masked=True),
        track_activations=True,
        track_auxiliary_gradients=True,
        input_shape = (1, 14, 14)
    ).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

print("This model has {} effective parameters.".format(model.parameter_count(masked = True)))
print("The conversion factor of this model is {} after layer {}.".format(model.conversion_factor, model.conversion_layer))

This model has 15538 effective parameters.
The conversion factor of this model is 4 after layer 2.


Let's get a dataset and define standard training and testing functions.

In [3]:
dataset = datasets.MNIST('../data', train=True, download=True,
                     transform=transforms.Compose([ 
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            transforms.Resize((14,14))
                        ]))
train_set, val_set = torch.utils.data.random_split(dataset, lengths=[int(0.9*len(dataset)), int(0.1*len(dataset))])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            transforms.Resize((14,14))
                        ])),
    batch_size=128, shuffle=True)

def train(model, train_loader, optimizer, criterion, epochs=10, val_loader=None):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        if val_loader is not None:
            print("Validation: ", end = "")
            test(model, val_loader, criterion)

def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

We'll pretrain the model before changing its architecture.

In [4]:
train(model, train_loader, optimizer, criterion, epochs=5, val_loader=val_loader)

Train Epoch: 0 [0/54000 (0%)]	Loss: 2.310916
Train Epoch: 0 [12800/54000 (24%)]	Loss: 2.199961
Train Epoch: 0 [25600/54000 (47%)]	Loss: 1.728310
Train Epoch: 0 [38400/54000 (71%)]	Loss: 1.394315
Train Epoch: 0 [51200/54000 (95%)]	Loss: 1.246807
Validation: Average loss: 0.0099, Accuracy: 3391/6000 (56.52%)
Train Epoch: 1 [0/54000 (0%)]	Loss: 1.293340
Train Epoch: 1 [12800/54000 (24%)]	Loss: 1.089069
Train Epoch: 1 [25600/54000 (47%)]	Loss: 0.802971
Train Epoch: 1 [38400/54000 (71%)]	Loss: 0.834708
Train Epoch: 1 [51200/54000 (95%)]	Loss: 0.709585
Validation: Average loss: 0.0068, Accuracy: 4036/6000 (67.27%)
Train Epoch: 2 [0/54000 (0%)]	Loss: 0.682295
Train Epoch: 2 [12800/54000 (24%)]	Loss: 0.808546
Train Epoch: 2 [25600/54000 (47%)]	Loss: 0.623847
Train Epoch: 2 [38400/54000 (71%)]	Loss: 0.658601
Train Epoch: 2 [51200/54000 (95%)]	Loss: 0.809328
Validation: Average loss: 0.0049, Accuracy: 4580/6000 (76.33%)
Train Epoch: 3 [0/54000 (0%)]	Loss: 0.618186
Train Epoch: 3 [12800/54000 (24

Let's see how we can use NeurOps to optimize the model.

First, we use a heuristic from `metrics.py` to measure the existing channels and neurons to determine which ones to prune. The simplest one is measuring the norm of incoming weights to a neuron. We'll copy the model (so we have access to the original), then score each neuron and prune the lowest scoring ones within each layer. After running the following block, try uncommenting different lines to see how different metrics affect the model.

In [5]:
modded_model = copy.deepcopy(model)
modded_optimizer = torch.optim.SGD(modded_model.parameters(), lr=0.01)
modded_optimizer.load_state_dict(optimizer.state_dict())

for i in range(len(model)-1):
    scores = weight_sum(model[i].weight)
    # scores = weight_sum(model[i].weight) +  weight_sum(model[i+1].weight, fanin=False, conversion_factor=model.conversion_factor if i == model.conversion_layer else -1)
    # scores = activation_variance(model.activations[str(i)])
    # scores = svd_score(model.activations[str(i)])
    # scores = nuclear_score(model.activations[str(i)], average=i<3)
    # Before trying this line, run the following block: # scores = fisher_info(mask_grads[i])
    print("Layer {} scores: mean {:.3g}, std {:.3g}, min {:.3g}, smallest 25%:".format(i, scores.mean(), scores.std(), scores.min()), end=" ")
    to_prune = np.argsort(scores.detach().numpy())[:int(0.25*len(scores))]
    print(to_prune)
    modded_model.prune(i, to_prune, optimizer=modded_optimizer, clear_activations=True)
print("The pruned model has {} effective parameters.".format(modded_model.parameter_count(masked = True)))
print("Validation after pruning: ", end = "")
test(modded_model, val_loader, criterion)
train(modded_model, train_loader, modded_optimizer, criterion, epochs=2, val_loader=val_loader)

Layer 0 scores: mean 4.01, std 0.291, min 3.7, smallest 25%: [6 2]
Layer 1 scores: mean 10.9, std 0.592, min 10.2, smallest 25%: [ 4  5  9 10]
Layer 2 scores: mean 11.9, std 0.921, min 10.6, smallest 25%: [ 2  5  8 10]
Layer 3 scores: mean 4.44, std 0.42, min 3.62, smallest 25%: [27 24 30 19 14  1  9 26]
The pruned model has 8914 effective parameters.
Validation after pruning: Average loss: 0.0079, Accuracy: 3952/6000 (65.87%)
Train Epoch: 0 [0/54000 (0%)]	Loss: 1.396421
Train Epoch: 0 [12800/54000 (24%)]	Loss: 0.634928
Train Epoch: 0 [25600/54000 (47%)]	Loss: 0.366687
Train Epoch: 0 [38400/54000 (71%)]	Loss: 0.492258
Train Epoch: 0 [51200/54000 (95%)]	Loss: 0.281330
Validation: Average loss: 0.0030, Accuracy: 5180/6000 (86.33%)
Train Epoch: 1 [0/54000 (0%)]	Loss: 0.261281
Train Epoch: 1 [12800/54000 (24%)]	Loss: 0.302074
Train Epoch: 1 [25600/54000 (47%)]	Loss: 0.503342
Train Epoch: 1 [38400/54000 (71%)]	Loss: 0.276111
Train Epoch: 1 [51200/54000 (95%)]	Loss: 0.392927
Validation: Aver

In [6]:
def collect_mask_grads():
    mask_grads = []
    for i in range(len(model.activations)-1):
        mask_grads.append(torch.empty(0, *model[i].mask_vector.shape))
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        for i in range(len(model.activations)-1):
            mask_grads[i] = torch.cat([mask_grads[i], model[i].mask_vector.grad.unsqueeze(0)])
    return mask_grads

#mask_grads = collect_mask_grads()


We can also try iterative pruning.

In [7]:
modded_model_iterative = copy.deepcopy(model)
modded_optimizer_iterative = torch.optim.SGD(modded_model_iterative.parameters(), lr=0.01)
modded_optimizer_iterative.load_state_dict(optimizer.state_dict())

for iter in range(5):
    for i in range(len(modded_model_iterative)-1):
        scores = weight_sum(modded_model_iterative[i].weight)
        # scores = weight_sum(model[i].weight) +  weight_sum(model[i+1].weight, fanin=False, conversion_factor=model.conversion_factor if i == model.conversion_layer else -1)
        # scores = activation_variance(model.activations[str(i)])
        # scores = svd_score(model.activations[str(i)])
        # scores = nuclear_score(model.activations[str(i)], average=i<3)
        print("Layer {} scores: mean {:.3g}, std {:.3g}, min {:.3g}, smallest 15%:".format(i, scores.mean(), scores.std(), scores.min()), end=" ")
        to_prune = np.argsort(scores.detach().numpy())[:int(0.15*len(scores))]
        print(to_prune)
        modded_model_iterative.prune(i, to_prune, optimizer=modded_optimizer_iterative, clear_activations=True)
    print("The pruned model now has {} effective parameters.".format(modded_model_iterative.parameter_count(masked = True)))
    print("Validation after pruning: ", end = "")
    test(modded_model_iterative, val_loader, criterion)
    train(modded_model_iterative, train_loader, modded_optimizer_iterative, criterion, epochs=2, val_loader=val_loader)

Layer 0 scores: mean 4.01, std 0.291, min 3.7, smallest 15%: [6]
Layer 1 scores: mean 9.6, std 0.599, min 8.88, smallest 15%: [5 4]
Layer 2 scores: mean 10.5, std 0.856, min 9.25, smallest 15%: [5 2]
Layer 3 scores: mean 3.95, std 0.383, min 3.3, smallest 15%: [27 19 24  1]
The pruned model now has 12008 effective parameters.
Validation after pruning: Average loss: 0.0037, Accuracy: 4966/6000 (82.77%)
Train Epoch: 0 [0/54000 (0%)]	Loss: 1.065550
Train Epoch: 0 [12800/54000 (24%)]	Loss: 0.311489
Train Epoch: 0 [25600/54000 (47%)]	Loss: 0.347963
Train Epoch: 0 [38400/54000 (71%)]	Loss: 0.363400
Train Epoch: 0 [51200/54000 (95%)]	Loss: 0.316475
Validation: Average loss: 0.0028, Accuracy: 5206/6000 (86.77%)
Train Epoch: 1 [0/54000 (0%)]	Loss: 0.255610
Train Epoch: 1 [12800/54000 (24%)]	Loss: 0.179851
Train Epoch: 1 [25600/54000 (47%)]	Loss: 0.273494
Train Epoch: 1 [38400/54000 (71%)]	Loss: 0.214253
Train Epoch: 1 [51200/54000 (95%)]	Loss: 0.369089
Validation: Average loss: 0.0027, Accuracy

Of course, we can also grow the model. The following cell uses a neurogenesis strategy similar to NORTH-Random.

In [8]:
modded_model_grow = copy.deepcopy(model)
modded_optimizer_grow = torch.optim.SGD(modded_model_grow.parameters(), lr=0.01)
modded_optimizer_grow.load_state_dict(optimizer.state_dict())

for iter in range(5):
    for i in range(len(modded_model_grow)-1):
        #score = orthogonality_gap(modded_model_grow.activations[str(i)])
        max_rank = modded_model_grow[i].out_features if i > modded_model_grow.conversion_layer else modded_model_grow[i].out_channels
        score = effective_rank(modded_model_grow.activations[str(i)])
        to_add = max(score-int(0.95*max_rank), 0)
        print("Layer {} score: {}/{}, neurons to add: {}".format(i, score, max_rank, to_add))
        modded_model_grow.grow(i, to_add, fanin_weights="iterative_orthogonalization", 
                               optimizer=modded_optimizer_grow)
    print("The grown model now has {} effective parameters.".format(modded_model_grow.parameter_count(masked = True)))
    print("Validation after growing: ", end = "")
    test(modded_model_grow, val_loader, criterion)
    train(modded_model_grow, train_loader, modded_optimizer_grow, criterion, epochs=2, val_loader=val_loader)

Layer 0 score: 8/8, neurons to add: 1
Layer 1 score: 16/16, neurons to add: 1
Layer 2 score: 16/16, neurons to add: 1
Layer 3 score: 32/32, neurons to add: 2


RuntimeError: shape '[34, 68]' is invalid for input of size 2176

Let's try masking neurons.

In [ ]:
modded_model_masked = copy.deepcopy(model)
modded_optimizer_masked = torch.optim.SGD(modded_model_masked.parameters(), lr=0.01)
modded_optimizer_masked.load_state_dict(optimizer.state_dict())

for i in range(len(modded_model_masked)-1):
    neurons = modded_model_masked[i].out_features if i > modded_model_masked.conversion_layer else modded_model_masked[i].out_channels
    modded_model_masked.mask(i, list(range(neurons//2, neurons)))

for iter in range(1):
    for i in range(len(modded_model_masked)-1):
        score = effective_rank(modded_model_masked.activations[str(i)])
        to_add = max(score-int(0.95*max_rank), 0)
        print("Layer {} score: {}/{}, neurons to add: {}".format(i, score, max_rank, to_add))
        modded_model_masked.grow(i, to_add, fanin_weights="iterative_orthogonalization", 
                                 optimizer=modded_optimizer_masked, mask=True)
    print("The grown model now has {} effective parameters.".format(modded_model_masked.parameter_count(masked = True)))
    print("Validation after growing: ", end = "")
    test(modded_model_masked, val_loader, criterion)
    train(modded_model_masked, train_loader, modded_optimizer_masked, criterion, epochs=2, val_loader=val_loader)
    